In [1]:
%reset
from sympy import *
import numpy as np

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
#Masses
motorMass     = 0.0273 #kg
frameMass     = 0.133 #kg
batteryMass   = 0.265 #kg
computerMass  = 0.059 #kg
propellerMass = 0.0034 #kg
radioMass     = 0.0005 #kg

#Frame Parameters
pxToM         = (7 / 18) / 1000 # m / px
length        = (517 - 143) * pxToM #m
width         = (390 - 311) * pxToM #m
armLength     = sqrt((294 - 96)**2 + (375 - 522)**2)  * pxToM #m
armWidth      = sqrt((213 - 231)**2 + (417 - 435)**2)  * pxToM #m
center        = Matrix([350, 333, 0])
 
armArea       = armLength * armWidth
centerArea    = length * width
totalArea     = armArea*4 + centerArea*2

centerMass    = frameMass * (2 * centerArea / totalArea)
armMass       = frameMass * (armArea / totalArea)

Oframeplate1_R = Matrix([0,0,0])
Oframeplate2_R = Matrix([0,0, 0.0023])

#Motor Parameters
motorDiameter= 0.0023 #m
motorHeight  = 0.00154 #m

motor_R      = ((Matrix([134, 157, 0]) - center) * pxToM).norm()
motor_Dir    = ((Matrix([134, 157, 0]) - center) * pxToM)/motor_R
Omotor_R1     = Matrix([motor_R,0,0])
Omotor_R2     = Omotor_R1.cross(Matrix([0,0,1]))
Omotor_R3     = Omotor_R2.cross(Matrix([0,0,1]))
Omotor_R4     = Omotor_R3.cross(Matrix([0,0,1]))
motor_Ang    = -acos(motor_Dir.dot(Matrix([1,0,0])))
rot          = Matrix([[cos(motor_Ang),-sin(motor_Ang),0],[sin(motor_Ang),cos(motor_Ang),0],[0,0,1]])

#Battery Parameters
batLength    = 0.0085 #m
batWidth     = 0.0045 #m
batHeight    = 0.0038 #m

Obat_R        = rot @ ((Matrix([350, 398, 0]) - center) * pxToM + Matrix([0,0,batHeight/2]))

#Computer Parameters
compLength   = 0.00486 #m
compWidth    = 0.00466 #m
compHeight   = 0.0026  #m

Ocomp_R       = Matrix([0,0,compHeight/2])

#Propeller Paramaters
spokeLength  = 0.13 #m
spokeMass    = propellerMass / 3

Oprop_R1     = Omotor_R1 + Matrix([0,0,motorHeight])
Oprop_R2     = Omotor_R2 + Matrix([0,0,motorHeight])
Oprop_R3     = Omotor_R3 + Matrix([0,0,motorHeight])
Oprop_R4     = Omotor_R4 + Matrix([0,0,motorHeight])

#Radio Parameter 
Oradio_R      = rot @ ((Matrix([350, 580, 0]) - center) * pxToM + Oframeplate2_R)

In [3]:
#Center of Mass

m_total      = motorMass*4 + frameMass + batteryMass + computerMass + propellerMass*4 + radioMass

weightAve    = (motorMass*Omotor_R1 + motorMass*Omotor_R2 + motorMass*Omotor_R3 + motorMass*Omotor_R4 + (centerMass/2 + armMass*4)*Oframeplate1_R + (centerMass/2)*Oframeplate2_R + 
                batteryMass*Obat_R + computerMass*Ocomp_R + propellerMass*Oprop_R1 + propellerMass*Oprop_R2 + propellerMass*Oprop_R3 + propellerMass*Oprop_R4 + radioMass*Oradio_R )

COM          = (weightAve / m_total).evalf()

motor_R1     = Omotor_R1      - COM
motor_R2     = Omotor_R2      - COM
motor_R3     = Omotor_R3      - COM
motor_R4     = Omotor_R4      - COM

frameplate1_R= Oframeplate1_R - COM
frameplate2_R= Oframeplate2_R - COM

bat_R        = Obat_R         - COM

comp_R       = Ocomp_R        - COM

prop_R1      = Oprop_R1       - COM
prop_R2      = Oprop_R2       - COM
prop_R3      = Oprop_R3       - COM
prop_R4      = Oprop_R4       - COM

radio_R      = Oradio_R       - COM


In [4]:
#Moment Of Interia Point Mass Approximation

MI_list = [[0,0,0],[0,0,0],[0,0,0]]


for i in range(3):
    for j in range(3):
        if (i != j):
            MI_list[i][j] = -(motorMass*motor_R1[i]*motor_R1[j] + motorMass*motor_R2[i]*motor_R2[j] + motorMass*motor_R3[i]*motor_R3[j] + motorMass*motor_R4[i]*motor_R4[j] + 
                             (centerMass/2 + armMass*4)*frameplate1_R[i]*frameplate1_R[j] + (centerMass/2)*frameplate2_R[i]*frameplate2_R[j] + batteryMass*bat_R[i]*bat_R[j] + 
                              computerMass*comp_R[i]*comp_R[j] + propellerMass*prop_R1[i]*prop_R1[j] + propellerMass*prop_R2[i]*prop_R2[j] + propellerMass*prop_R3[i]*prop_R3[j] +
                              propellerMass*prop_R4[i]*prop_R4[j] + radioMass*radio_R[i]*radio_R[j] )


MI_list[0][0] = (motorMass*(motor_R1[1]**2+motor_R1[2]**2) + motorMass*(motor_R2[1]**2+motor_R2[2]**2) + motorMass*(motor_R3[1]**2+motor_R3[2]**2) + motorMass*(motor_R4[1]**2+motor_R4[2]**2) + 
                             (centerMass/2 + armMass*4)*(frameplate1_R[1]**2+frameplate1_R[2]**2) + (centerMass/2)*(frameplate2_R[1]**2+frameplate2_R[2]**2) + batteryMass*(bat_R[1]**2+bat_R[2]**2) + 
                              computerMass*(comp_R[1]**2+comp_R[2]**2) + propellerMass*(prop_R1[1]**2+prop_R1[2]**2) + propellerMass*(prop_R2[1]**2+prop_R2[2]**2) + propellerMass*(prop_R3[1]**2+prop_R3[2]**2) +
                              propellerMass*(prop_R4[1]**2+prop_R4[2]**2) + radioMass*(radio_R[1]**2+radio_R[2]**2) )

MI_list[1][1] = (motorMass*(motor_R1[0]**2+motor_R1[2]**2) + motorMass*(motor_R2[0]**2+motor_R2[2]**2) + motorMass*(motor_R3[0]**2+motor_R3[2]**2) + motorMass*(motor_R4[0]**2+motor_R4[2]**2) + 
                             (centerMass/2 + armMass*4)*(frameplate1_R[0]**2+frameplate1_R[2]**2) + (centerMass/2)*(frameplate2_R[0]**2+frameplate2_R[2]**2) + batteryMass*(bat_R[0]**2+bat_R[2]**2) + 
                              computerMass*(comp_R[0]**2+comp_R[2]**2) + propellerMass*(prop_R1[0]**2+prop_R1[2]**2) + propellerMass*(prop_R2[0]**2+prop_R2[2]**2) + propellerMass*(prop_R3[0]**2+prop_R3[2]**2) +
                              propellerMass*(prop_R4[0]**2+prop_R4[2]**2) + radioMass*(radio_R[0]**2+radio_R[2]**2) )

MI_list[2][2] = (motorMass*(motor_R1[0]**2+motor_R1[1]**2) + motorMass*(motor_R2[0]**2+motor_R2[1]**2) + motorMass*(motor_R3[0]**2+motor_R3[1]**2) + motorMass*(motor_R4[0]**2+motor_R4[1]**2) + 
                             (centerMass/2 + armMass*4)*(frameplate1_R[0]**2+frameplate1_R[1]**2) + (centerMass/2)*(frameplate2_R[0]**2+frameplate2_R[1]**2) + batteryMass*(bat_R[0]**2+bat_R[1]**2) + 
                              computerMass*(comp_R[0]**2+comp_R[1]**2) + propellerMass*(prop_R1[0]**2+prop_R1[1]**2) + propellerMass*(prop_R2[0]**2+prop_R2[1]**2) + propellerMass*(prop_R3[0]**2+prop_R3[1]**2) +
                              propellerMass*(prop_R4[0]**2+prop_R4[1]**2) + radioMass*(radio_R[0]**2+radio_R[1]**2) )

MI = Matrix(MI_list).evalf()

In [5]:
t = symbols("t")
phi = Function('phi')(t)
theta = Function('theta')(t)
psi = Function('psi')(t)
R1 = Matrix([[cos(phi),-sin(phi),0],[sin(phi),cos(phi),0],[0,0,1]])
R2 = Matrix([[1,0,0],[0,cos(theta),-sin(theta)],[0,sin(theta),cos(theta)]])
R3 = Matrix([[cos(psi),-sin(psi),0],[sin(psi),cos(psi),0],[0,0,1]])

R = R3 @ R2 @ R1

In [6]:
m_total

0.5802999999999999

In [7]:
COM

Matrix([
[ 0.00734390711247691],
[-0.00901297691076712],
[ 0.00122287202080556]])

In [8]:
print(MI)

Matrix([[0.000778741496230108, 4.67668921063893e-5, -2.89783065857405e-6], [4.67668921063893e-5, 0.000759452121539762, 3.55642853552270e-6], [-2.89783065857405e-6, 3.55642853552270e-6, 0.00153725292232051]])


In [9]:
motor_R1

Matrix([
[   0.101010335883857],
[ 0.00901297691076712],
[-0.00122287202080556]])

In [10]:
bat_R

Matrix([
[ 0.00862338137523516],
[ -0.0105832407786977],
[0.000677127979194443]])

In [11]:
#Torque calc
kv      = 1750 #rpm/v from motor
V       = 22.2 #v from battery
Amax    = 55   #from ESC
kt      = ((kv*2*np.pi)/60) #https://www.tytorobotics.com/blogs/articles/how-to-calculate-electric-motor-torque

C       = (kv*V*2*np.pi)/60 # max rad/s
Tmax    = kt*Amax
B       = C/Tmax

Ct      = 1       #Thrust coeffecient need to find
rho     = 1.293   #kg/m3 atmo density
D       = 0.12954 #m for 5.1 inch rotors
def Tmotor(F):
    rads = 2*np.pi*np.sqrt((F)/(Ct*rho*D**4))
    T    = (rads - C)/(-B)
    return T

In [12]:
#motor toque for pure hover
g = 9.81
fPermotor = (m_total*g)/4
Tmotor(fPermotor)


9106.051268099282

In [13]:
#motor toque and angle for circle
g = 9.81
altF = (m_total*g)/4

v = 0.5
r = 2

circF = ((v**2/r)*m_total)/4

Force  = np.array([circF,altF])
T_ = Tmotor(np.linalg.norm(Force))
phiR    = -atan(Force[0]/Force[1])

interial_rot = Matrix([0,0,v/r])

rot = R.subs(theta,0).subs(psi,0).subs(phi,phiR) @ interial_rot

print(np.linalg.norm(Force))

q0 = cos(phiR/2) 
q1 = sin(phiR/2)
q2 = 0
q3 = 0


print(Force)
print(T_)
print(phiR)
print(q0)
print(q1)
print(rot)

1.423301280319702
[0.01813437 1.42318575]
9106.011767024882
-0.0127414103580744
0.999979707126395
-0.00637066208567390
Matrix([[0], [0], [0.250000000000000]])


In [14]:
#Complex

In [15]:
#quaternian calc
def Q(phi1,theta1,psi1):
    q0_ = cos(phi1/2)*cos(theta1/2)*cos(psi1/2) + sin(phi1/2)*sin(theta1/2)*sin(psi1/2)
    q1_ = sin(phi1/2)*cos(theta1/2)*cos(psi1/2) - cos(phi1/2)*sin(theta1/2)*sin(psi1/2)
    q2_ = cos(phi1/2)*sin(theta1/2)*cos(psi1/2) + sin(phi1/2)*cos(theta1/2)*sin(psi1/2)
    q3_ = cos(phi1/2)*cos(theta1/2)*sin(psi1/2) - sin(phi1/2)*sin(theta1/2)*cos(psi1/2)
    return Matrix([q0_,q1_,q2_,q3_])

In [16]:

g = 9.81
a = g + 1
fPermotor = (m_total*a)/4
Tmotor(fPermotor)

9057.651053430598

In [17]:
g = 9.81
a = g - 1
fPermotor = (m_total*a)/4
Tmotor(fPermotor)

9156.987960233548

In [18]:

g = 9.81
a = g + 5
fPermotor = (m_total*a)/4
Tmotor(fPermotor)

8883.482473151025

In [19]:
g = 9.81
a = g - 5
fPermotor = (m_total*a)/4
Tmotor(fPermotor)

9397.79917849867

In [20]:
Q(0,0,np.pi/2)

Matrix([
[0.707106781186548],
[                0],
[                0],
[0.707106781186547]])

In [21]:
-1.56789238706824+np.pi

1.5737002665215531

In [22]:
-1.56789238706824+np.pi

1.5737002665215531

In [23]:
np.pi/2

1.5707963267948966

In [24]:
0.707106781186547/2

0.3535533905932735

In [25]:
1.5737002665215531-1.5707963267948966


0.0029039397266565903

In [26]:
Q(0,0.25,np.pi/2)

Matrix([
[  0.701589698775332],
[-0.0881583494193193],
[ 0.0881583494193194],
[  0.701589698775332]])

In [29]:
0.707106781186547-0.706515403154211

0.0005913780323359985